<a href="https://colab.research.google.com/github/shinnew9/Apziva_practice_code/blob/main/Project3-PotentialTalents/DeepSeek_r1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Work in 2 process:
1. Rank: The fit column is empty, and we should calculate the similarity of the search term against the job titles of the candidates using cosine similarity.
2. Design and Learn how to rank: We need labels for the 2nd project.
- Pick a few candidates and manually overwrite their fit score (cosine_similarity). I should pick one among the word embedding methods I've gon through so far.
- Train a learning to rank model in particular, 'RankNet' in PyTorch (<- they have many examples on github in code). Rank Net use pair-wise ranking, I need to transform fit score by selecting to candidates randomly and comparing their fit scores if one of the fit score is larger than the pair, it is transformed as 1. if it's smaller than the other pair, the label becomes 0, indicating that one candidate should be ranked above the other candidates.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Package Installation - SOTA

In [2]:
!pip install huggingface_hub

In [3]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3


In [4]:
!pip install transformers accelerate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Open CSV

In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Apziva/3rd_PotentialTalents/data.csv')
df_copy = df.copy()
df_copy.head(5)

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


### DeepSeek

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) yes
Token is valid (permission: fineGrained).
The token `Apziva_practice` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-aut

In [9]:
job_titles = df_copy['job_title'].dropna().astype(str).tolist()
print(job_titles[:5])

['2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional', 'Native English Teacher at EPIK (English Program in Korea)', 'Aspiring Human Resources Professional', 'People Development Coordinator at Ryan', 'Advisory Board Member at Celal Bayar University']


In [10]:
# # !nvidia-smi
# !nvida-smi | grep"H100"

In [11]:
# Step 3: Load deepeseek-r1 Model & Tokenizer
model_name = "deepseek-ai/deepseek-coder-6.7b-instruct" # "deepseek-ai/deepseek-r1"   # change this if needed
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype = torch.float16,  # float16 for coder, float32 for r1
    device_map="auto",
    # quantization_config = {"quant_method":"bitsandbytes_8bit"},
    trust_remote_code=True)


# "deepseek-ai/deepseek-r1"doesn't work in here b/c, it's using FP8(float 8-bit precision), which is not supported in Google Colab.
# It does not have NVIDIA H100 GPUs (which support FP8), so the model can't be loaded

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [12]:
# job_titles를 하나의 문자열로 변환(줄바꿈 추가)
job_titles_text = "\n".join([f"{idx}:{title}" for idx, title in enumerate(df_copy['job_title'][:30])])
print(job_titles_text[:10])

# 프롬프트 생성
prompt = f"""I have the following job titles (with index numbers):

{job_titles_text}

Please rank these job titles based on relevance for an aspiring human resources professional.
Provide the output in a structured format like:
1. [index] - [Best job title]
2. [index] - [Second best job title]
3. [index] - [Third best job title]

Also, provide the output in **JSON format** like:
{{
    "rankings": [
        {{"rank": 1, "index": X, "job_title": "[Best job title]"}},
        {{"rank": 2, "index": Y, "job_title": "[Second best job title]"}},
        {{"rank": 3, "index": Z, "job_title": "[Third best job title]"}}
    ]
}}
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Move to GPU if available
outputs = model.generate(**inputs, max_new_tokens=500)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("+Model Output:\n", result)




import json

# 🔹 모델이 생성한 JSON 결과 파싱
try:
    rankings_json = json.loads(result)
    ranked_indices = [entry["index"] for entry in rankings_json["rankings"]]

    # 🔹 원본 데이터에서 정렬된 순위로 보기
    ranked_df = df.iloc[ranked_indices]
    display(ranked_df)
except json.JSONDecodeError:
    print("Error: Model output is not a valid JSON.")

0:2019 C.T


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
import json

# 🔹 모델이 생성한 JSON 결과 파싱
try:
    rankings_json = json.loads(result)
    ranked_indices = [entry["index"] for entry in rankings_json["rankings"]]

    # 🔹 원본 데이터에서 정렬된 순위로 보기
    ranked_df = df.iloc[ranked_indices]
    display(ranked_df)
except json.JSONDecodeError:
    print("Error: Model output is not a valid JSON.")

Error: Model output is not a valid JSON.


### Extract only key-phrase & Analysis


In [ ]:
# Step 5-1: Extract only key-phrase from the Model Output
# The generated

import re

# Example model output
generated_text = """Extract key phrases from the following text: 'Aspiring Human Resources Professional looking for new opportunities in the field of HRM'

import re

text = 'Aspiring Human Resources Professional looking for new opportunities in the field of HRM'

# Use regular expression to find key phrases
key_phrases = re.findall(r'[A-Z][a-z]+ [A-Z][a-z]+ [" """


# Use regex to extract key phrases
key_phrases = re.findall(r"[A-Z][a-z]+(?: [A-Z][a-z]+)", generated_text)

print("Extracted Key Phrases: ", key_phrases)

Extracted Key Phrases:  ['Aspiring Human', 'Resources Professional', 'Aspiring Human', 'Resources Professional']


In [ ]:
# Step 5-2: Apply Key Phrase Extraction to All Job Titles
# To systematically extract key phrass from all job title,s apply the regex fun tion to each job title in the dataset.

# Extract key phrases for each job title
df_copy["ds-r1_key_phrases"]= df_copy["job_title"].apply(lambda x: re.findall(r"[A-Z][a-z]+(?: [A-Z][a-z]+)", x))

# display results
import pandas as pd

pd.set_option("display.max_rows", None)  # Show all rows
print(df_copy[["job_title", "ds-r1_key_phrases"]])

                                             job_title  \
0    2019 C.T. Bauer College of Business Graduate (...   
1    Native English Teacher at EPIK (English Progra...   
2                Aspiring Human Resources Professional   
3               People Development Coordinator at Ryan   
4      Advisory Board Member at Celal Bayar University   
5                  Aspiring Human Resources Specialist   
6    Student at Humber College and Aspiring Human R...   
7                                 HR Senior Specialist   
8    Student at Humber College and Aspiring Human R...   
9    Seeking Human Resources HRIS and Generalist Po...   
10                       Student at Chapman University   
11   SVP, CHRO, Marketing & Communications, CSR Off...   
12   Human Resources Coordinator at InterContinenta...   
13   2019 C.T. Bauer College of Business Graduate (...   
14   2019 C.T. Bauer College of Business Graduate (...   
15   Native English Teacher at EPIK (English Progra...   
16            

In [ ]:
# Step 5-3: Count Frequency of Key Phrases
# Analyze how frequently certain key phrases appear across job titles.

from collections import Counter

# Flatten all key phrases into a single list
all_key_phrases = [phrase for sublist in df_copy["ds-r1_key_phrases"] for phrase in sublist]

# Count occurrences
key_phrase_counts = Counter(all_key_phrases)

# Print the most common key phrases
print("Most Common Key Phrases: ", key_phrase_counts.most_common(20))

Most Common Key Phrases:  [('Aspiring Human', 27), ('Human Resources', 25), ('Resources Professional', 9), ('Seeking Human', 8), ('Bauer College', 7), ('Business Graduate', 7), ('Magna Cum', 7), ('Humber College', 7), ('Resources Generalist', 7), ('People Development', 6), ('Native English', 5), ('English Program', 5), ('Resources Specialist', 5), ('Senior Specialist', 5), ('Advisory Board', 4), ('Celal Bayar', 4), ('Generalist Positions', 4), ('Chapman University', 4), ('The Woodlands', 4), ('Continental Buckhead', 4)]
